In [1]:
!pip install ccxt

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [5]:
import ccxt
import pandas as pd
import time

# Инициализация биржи
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '4h'
since = exchange.parse8601('2017-08-17T00:00:00Z')  # Самая ранняя дата для BTC/USDT на Binance

# Список для хранения всех данных
all_ohlcv = []

# Текущая временная метка в миллисекундах
current_time = int(time.time() * 1000)  # Текущая дата в миллисекундах

# Ограничение на количество свечей за запрос (Binance обычно возвращает до 500 или 1000)
limit = 1000

# Загрузка данных с пагинацией
while since < current_time:
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break  # Если данных больше нет, выходим
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Обновляем since до следующей свечи
        print(f"Загружено {len(ohlcv)} свечей до {pd.to_datetime(since, unit='ms')}")
        time.sleep(1)  # Задержка, чтобы не превысить лимит запросов API
    except Exception as e:
        print(f"Ошибка: {e}")
        break

# Преобразование в DataFrame
df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Удаление дубликатов, если есть
df = df[~df.index.duplicated(keep='first')]

# Сохранение в CSV
df.to_csv('btc_usdt_4h_full.csv')
print(f"Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: {len(df)}")

Загружено 1000 свечей до 2018-01-30 16:00:00.001000
Загружено 1000 свечей до 2018-07-18 00:00:00.001000
Загружено 1000 свечей до 2018-12-31 20:00:00.001000
Загружено 1000 свечей до 2019-06-17 00:00:00.001000
Загружено 1000 свечей до 2019-11-30 20:00:00.001000
Загружено 1000 свечей до 2020-05-15 16:00:00.001000
Загружено 1000 свечей до 2020-10-29 08:00:00.001000
Загружено 1000 свечей до 2021-04-14 00:00:00.001000
Загружено 1000 свечей до 2021-09-27 16:00:00.001000
Загружено 1000 свечей до 2022-03-13 08:00:00.001000
Загружено 1000 свечей до 2022-08-27 00:00:00.001000
Загружено 1000 свечей до 2023-02-09 16:00:00.001000
Загружено 1000 свечей до 2023-07-26 08:00:00.001000
Загружено 1000 свечей до 2024-01-09 00:00:00.001000
Загружено 1000 свечей до 2024-06-23 16:00:00.001000
Загружено 1000 свечей до 2024-12-07 08:00:00.001000
Загружено 685 свечей до 2025-03-31 12:00:00.001000
Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: 16685


In [7]:
!pip install pandas_ta

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [11]:
import pandas as pd
import pandas_ta as ta

# Загрузка данных
df = pd.read_csv('btc_usdt_4h_full.csv', index_col='timestamp', parse_dates=True)

# Differencing
for k in range(1, 6):
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[f'{col}_diff_{k}'] = df[col] / df[col].shift(k)

# EBSW
ebsw_result = ta.ebsw(df['close'])
df['ebsw'] = ebsw_result

# Chaikin Money Flow
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)

# VWAP
df['tp'] = (df['low'] + df['high'] + df['close']) / 3
df['vwap'] = (df['tp'] * df['volume']).cumsum() / df['volume'].cumsum()
df['vwap_open'] = df['vwap'] / df['open']

# Other Ratios
df['high_low'] = df['high'] / df['low']
df['high_open'] = df['high'] / df['open']
df['close_open'] = df['close'] / df['open']
df['low_open'] = df['low'] / df['open']

# Time Information
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month

# Удаляем строки с NaN
df.dropna(inplace=True)

df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']

def classify(u, v):
    if u >= 0.0075:
        return 0  # c0
    elif v <= -0.0075:
        return 1  # c1
    else:
        return 2  # c2

df['target'] = [classify(u, v) for u, v in zip(df['u_t1'], df['v_t1'])]
df.dropna(inplace=True)  # Удаляем последнюю строку

# Сохранение обработанных данных
df.to_csv('btc_usdt_4h_with_target.csv')

/tmp/ipykernel_157/429230575.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ebsw_result = ta.ebsw(df['close'])


In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.fillna(method='ffill', inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной с новым порогом
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.005
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление ATR
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.005, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Проверка распределения
print("Распределение y_train:", np.bincount(y_train))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.2, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=128, n_a=128, n_steps=5, gamma=1.5,
    lambda_sparse=0.005, optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy'],
    max_epochs=300,
    patience=100,
    batch_size=512,
    virtual_batch_size=256,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

/tmp/ipykernel_258/1382881761.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Распределение y_train: [6147 2598  786]
Распределение y_val: [167  88   9]
Распределение y_test: [245 125  39]
Веса классов: {0: 1.0, 1: 1.2, 2: 2.0}
epoch 0  | loss: 2.00451 | val_0_accuracy: 0.54167 |  0:00:01s
epoch 1  | loss: 1.13647 | val_0_accuracy: 0.54167 |  0:00:02s
epoch 2  | loss: 1.07677 | val_0_accuracy: 0.5947  |  0:00:03s
epoch 3  | loss: 1.08395 | val_0_accuracy: 0.56818 |  0:00:04s
epoch 4  | loss: 0.99726 | val_0_accuracy: 0.61742 |  0:00:05s
epoch 5  | loss: 1.01899 | val_0_accuracy: 0.60606 |  0:00:06s
epoch 6  | loss: 1.00234 | val_0_accuracy: 0.56439 |  0:00:07s
epoch 7  | loss: 1.00071 | val_0_accuracy: 0.57197 |  0:00:08s
epoch 8  | loss: 0.97034 | val_0_accuracy: 0.5947  |  0:00:08s
epoch 9  | loss: 0.96269 | val_0_accuracy: 0.52273 |  0:00:09s
epoch 10 | loss: 0.99751 | val_0_accuracy: 0.61742 |  0:00:10s
epoch 11 | loss: 0.93318 | val_0_accuracy: 0.60985 |  0:00:11s
epoch 12 | loss: 0.93138 | val_0_accuracy: 0.64394 |  0:00:12s
epoch 13 | loss: 0.90137 | val_

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [395  14]
Распределение истинных классов: [245 125  39]
Accuracy: 0.5892
F1-score: 0.4638

Classification Report:
               precision    recall  f1-score   support

          c0       0.60      0.96      0.74       245
          c1       0.36      0.04      0.07       125
          c2       0.00      0.00      0.00        39

    accuracy                           0.59       409
   macro avg       0.32      0.33      0.27       409
weighted avg       0.47      0.59      0.46       409


Топ-5 важных признаков:
 high_low        0.148183
atr             0.117177
close_diff_1    0.093142
open_diff_1     0.088827
close_open      0.067836
dtype: float64


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
